<a href="https://colab.research.google.com/github/BrunoReis136/API_fin_powered_by_OpenAI/blob/main/projeto_fin_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USANDO API DA FINNHUB

## Definir Classe Cliente Finnhub

In [7]:
import requests
import time

# Classe Centralizadora das operações com a API Finnhub
class FinnhubClient:

  # Construtor da Classe recebendo a chave e definindo o url base
  def __init__(self, api_key):
    self.api_key = api_key                      # Chave da API
    self.base_url = 'https://finnhub.io/api/v1' # URL base do Finnhub


  # Definir Método de busca pelo API atravéd de requisição GET
  def _get(self, endpoint, params=None):
    # Define a variável params como um dicionário
    if params is None:
      params = {}

    # Insere no params a chave de api como valor da chave 'token'
    params["token"] = self.api_key

    # Executa a requisição usando a variável 'base_url' , 'endpoint' e passando os parametros pelo dicionário 'params'
    r = requests.get(f'{self.base_url}/{endpoint}', params = params)

    # Lança excessão caso a resposta não seja 200 (erro HTTP)
    r.raise_for_status()

    # Pausa na sequencia de comandos pra folga necessária em caso de limite de execuções do API (plano gratuito)
    time.sleep(0.3)

    # Retorna o resultado do formato JSON para dicionário python
    return r.json()



  # Define método para retorno das cotação atual do ativo definido (preço, máxima, mínima, abertura, fechamento anterior)
  def quote(self, symbol):
    return self._get(
        "quote",
        {"symbol":symbol} # Código do ativo (APPL, MSFT, TSLA)
    )

  # Define método pra retorno das candles de um ativo definido em determinado período
  def candles(self, symbol, resolution, start, end):
    return self._get(
        "stock/candle",
        {
            "symbol": symbol,         # Código do ativo (APPL, MSFT, TSLA)
            "resolution": resolution, # Resolução da vela : Em minutos > 1, 5, 15, 30, 60; Período > D (days), W (weeks), M (months)
            "from": start,            # Data inicial
            "to": end                 # Data final
        }
    )


  # Define método para retorno dos rendimentos do ativo definido (real, estimado, surpresa)
  def earnings(self, symbol):
    return self._get(
        "stock/earnings",
        {"symbol":symbol}
    )


  # Define método para retorno dos dados financeiros reportados (demonstração de resultados, balanço patrimonial, fluxo de caixa)
  def financials(self, symbol):
    return self._get(
        "stock/stock/financials-reported",
        {"symbol": symbol}
    )

## Funções de Normalização dos Dados

In [8]:
import pandas as pd


# Função de normalização dos Candles para DataFrame estruturado
def normalize_candles(data):
  # Confere se o campo "s" da resposta do API veio com valor diferente de "ok", que confirma se os dados estão válidos
  # Caso diferente, retorna um DataFrame vazio
  if data.get("s") != 'ok':
    return pd.DataFrame()

  # Cria um DataFrame a partir de cada array retornado pela API

  df = pd.Dataframe({
      "timestamp": data['t'], # Tempo em formato UNIX (epoch)
      "open":data['o'],       # Preço de abertura
      "high":data['h'],       # Preço máximo
      "low":data['l'],        # Preço mínimo
      "close":data['c'],      # Preço de fechamento
      "volume":data['v']      # Volume negociado
  })

  # Converte a coluna data para formato legível
  df['date'] = pd.to_datetime(df['timestamp'], unit='s')

  # Retorna o dataframe excluindo a coluna com tempo em formato bruto
  return df.drop(columns='timestamp')


# Função de normalização direta dos dados de rendimentos, nossa class já retorna em dicionário python, então é só converter pra dataframe
def normalize_earnings(data):
  df = pd.DataFrame(data)


# Função de normalização dos dados financeiros reportados
# Os dados vêm aninhados, então usamos o json_normalize
def normalize_financials(data):
  df = pd.json_normalize(
      data['data'],                 # Lista principal dos relatórios
      record_path=['record','ic'],  # Caminho até os indicadores financeiros
      meta=[
          ['report','period'],      # Período do relatório
          ['report','year'],        # Ano fiscal
          ['report','quarter']      # Trimestre fiscal
      ]
  )

  # Retorna o dataframe
  return df

## Gerando dados práticos para interpretação através do API da Openai

In [9]:
# Função que adiciona ao DataFrame colunas com valores baseados nos preços
def add_price_features(df):
  # Reorganiza os dados por ordem crescente de data
  df = df.sort_values("date")

  # Variação relativca do preço
  df['return_1d'] = df['close'].pct_change()

  # Média móvel do ativo de 7 períodos, tendência a curto prazo
  df['ma_7'] = df['close'].rolling(7).mean()

  # Média móvel do ativo de 21 períodos, tendência a curto prazo
  df['ma_21'] = df['close'].rolling(21).mean()

  # Volatilidade de 21 períodos
  df['volatility_21'] = df['return_1d'].rolling(21).std()

  # Retorna o DataFrame com os dados trabalhados
  return df


# Função que adiciona ao DataFrame colunas com valores baseados nos 'earnings per share' - 'lucro por ação'
def add_earning_features(df):
  # Diferença real entre lucros estiamdos e lucros reais, positivo = bom
  df['EPS_diff'] = df['actual'] - df['estimate']

  # Renomeia a coluna de Surpresa Estimada para uma melhor semântica / leitura pelo LLM
  df = df.rename(columns={"surprisePercent": "eps_surprise_pct"})

  return df

## Utilização dos recursos criados

In [ ]:
api_key = 'd53k8hhr01qkplgvn6ngd53k8hhr01qkplgvn6o0'
symbol = 'APPL'

client = FinnhubClient(api_key)

